# Part 2: Training a Linear model with Trainable Basis Functions

#### Welcome to the part 2 tutorial!

This tutorial will demonstrate how to utilise `nnAudio trainable basis functions` to build a powerful classifier in which the spectrograms are actually finetuned to the task at hand during backpropagation.

Same as previous tutorial, `Google SPEECHCOMMANDS dataset v2 (12 classes) with linear model` will be used.

[Step 1: experiment set up](#Step-1:-experiment-set-up)\
[Step 2: setting up nnAudio basis functions](#Step-2:-setting-up-nnAudio-basis-functions)\
[Step 3: setting up the lightning module](#Step-3:-setting-up-the-lightning-module)\
[Step 4: setting up the model ](#Step-4:-setting-up-the-model)\
[Step 5: training the model](#Step-5:-training-the-model)

## Step 1: experiment set up 
The code below is same as step 1 to 5 in part 1 tutorial.\
Feel free to check back for the code detail and the explanation.

In [ ]:
# import related libraries
import torch
from torch import Tensor
import torchaudio 
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import WeightedRandomSampler,DataLoader
import torch.optim as optim
from AudioLoader.Speech import SPEECHCOMMANDS_12C #for 12 classes KWS task
from pytorch_lightning import Trainer
from pytorch_lightning.core.lightning import LightningModule
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from nnAudio.features.mel import MelSpectrogram, STFT

# setting up configuration
device = 'cuda:0'
gpus = 1
batch_size= 100
max_epochs = 200
check_val_every_n_epoch = 2
num_sanity_val_steps = 5
data_root= './' # Download the data here
download_option= False
n_mels= 40 
input_dim= (n_mels*101)
output_dim= 12

# setting up dataset
trainset = SPEECHCOMMANDS_12C(root=data_root,
                              url='speech_commands_v0.02',
                              folder_in_archive='SpeechCommands',
                              download= download_option,subset= 'training') 

validset = SPEECHCOMMANDS_12C(root=data_root,
                              url='speech_commands_v0.02',
                              folder_in_archive='SpeechCommands',
                              download= download_option,subset= 'validation')

#data rebalancing
class_weights = [1,1,1,1,1,1,1,1,1,1,4.6,1/17]
sample_weights = [0] * len(trainset)
for idx, (data,rate,label,speaker_id, _) in enumerate(trainset):
    class_weight = class_weights[label]
    sample_weights[idx] = class_weight
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights),replacement=True)

#Data processing
def data_processing(data):
    waveforms = []
    labels = []    
    for batch in data:
        waveforms.append(batch[0].squeeze(0)) #after squeeze => (audio_len) tensor # remove batch dim
        labels.append(batch[2])             
    waveform_padded = nn.utils.rnn.pad_sequence(waveforms, batch_first=True)     
    output_batch = {'waveforms': waveform_padded, 
             'labels': torch.tensor(labels),
             }
    return output_batch

# data loading
trainloader = DataLoader(trainset,                                
                              collate_fn=lambda x: data_processing(x),
                                         batch_size=batch_size,sampler=sampler)

validloader = DataLoader(validset,                               
                              collate_fn=lambda x: data_processing(x),
                                         batch_size=batch_size)

# Step 2: setting up nnAudio basis functions
## nnAudio guideline for trainable basis functions 
```
nnAudio.features.mel.MelSpectrogram(trainable_mel= ,trainable_STFT=) 
```
The function above is controlling if mel bases and STFT trainable

* A. Both Mel and STFT are non-trainable: 
`trainable_mel=False, trainable_STFT=False`
* B. Mel is trainable while STFT is fixed: 
`trainable_mel=True, trainable_STFT=False`
* C. Mel is fixed while STFT is trainable: 
`trainable_mel=False, trainable_STFT=True`
* D. Both Mel and STFT are trainable:
`trainable_mel=True, trainable_STFT=True`

Mel-spectrogram is used as an example.You can modify Mel-spectrogram argument from the function below: 

In [ ]:
mel_layer = MelSpectrogram(sr=16000, 
                           n_fft=480,
                           win_length=None,
                           n_mels=n_mels, 
                           hop_length=160,
                           window='hann',
                           center=True,
                           pad_mode='reflect',
                           power=2.0,
                           htk=False,
                           fmin=0.0,
                           fmax=None,
                           norm=1,
                           trainable_mel=True,
                           trainable_STFT=True,
                           verbose=True)

## Step 3: setting up the lightning module

In [ ]:
class SpeechCommand(LightningModule):
    def training_step(self, batch, batch_idx):
        outputs, spec = self(batch['waveforms']) 
        #return outputs [2D] for calculate loss, return spec [3D] for visual
        loss = self.criterion(outputs, batch['labels'].long())

        acc = sum(outputs.argmax(-1) == batch['labels'])/outputs.shape[0] #batch wise
        
        self.log('Train/acc', acc, on_step=False, on_epoch=True)
        self.log('Train/Loss', loss, on_step=False, on_epoch=True)
        #log(graph title, take acc as data, on_step: plot every step, on_epch: plot every epoch)
        return loss

     
    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx,
                       optimizer_closure, on_tpu, using_native_amp, using_lbfgs):
        
        optimizer.step(closure=optimizer_closure)
        with torch.no_grad():
            torch.clamp_(self.mel_layer.mel_basis, 0, 1)
        #after optimizer step, do clamp function on mel_basis         

        
    def validation_step(self, batch, batch_idx):               
        outputs, spec = self(batch['waveforms'])
        loss = self.criterion(outputs, batch['labels'].long())        
       
        self.log('Validation/Loss', loss, on_step=False, on_epoch=True)                     
        output_dict = {'outputs': outputs,
                       'labels': batch['labels']}        
        return output_dict

    
    def validation_epoch_end(self, outputs):
        pred = []
        label = []
        for output in outputs:
            pred.append(output['outputs'])
            label.append(output['labels'])
        label = torch.cat(label, 0)
        pred = torch.cat(pred, 0)
        acc = sum(pred.argmax(-1) == label)/label.shape[0]
        
        self.log('Validation/acc', acc, on_step=False, on_epoch=True)    

    
    def configure_optimizers(self):
        model_param = []
        for name, params in self.named_parameters():
            if 'mel_layer.' in name:
                pass
            else:
                model_param.append(params)          

        optimizer = optim.SGD(model_param, lr=1e-3, momentum= 0.9, weight_decay= 0.001)
        return [optimizer]

## Step 4: setting up the model

In [ ]:
class Linearmodel_nnAudio(SpeechCommand):
    def __init__(self): 
        super().__init__()
        self.mel_layer = mel_layer       
        self.criterion = nn.CrossEntropyLoss()
        self.linearlayer = nn.Linear(input_dim, output_dim)
      
    def forward(self, x): 
        #x: 2D [B, 16000]
        spec = self.mel_layer(x)  
        #spec: 3D [B, F40, T101]
        
        spec = torch.log(spec+1e-10)
        flatten_spec = torch.flatten(spec, start_dim=1) 
        #flatten_spec: 2D [B, F*T(40*101)] 
        #start_dim: flattening start from 1st dimention
        
        out = self.linearlayer(flatten_spec) 
        #out: 2D [B,number of class(12)] 
                               
        return out, spec 

model_nnAudo = Linearmodel_nnAudio()
model_nnAudo = model_nnAudo.to(device)

## Step 5: training the model

The trained model weight will be saved in `lightning_logs` folder.

In [ ]:
trainer = Trainer(gpus=gpus, max_epochs=max_epochs,
    check_val_every_n_epoch= check_val_every_n_epoch,
    num_sanity_val_steps=num_sanity_val_steps)

trainer.fit(model_nnAudo, trainloader, validloader)

# Congratulations! This is the end of Part 2 tutorial.

Next step, you can evaluate the model performance in part 3 tutorial with the trained model weight.
